## **efficient net + triplet**

구조 커스텀에 용이하기 위해 파이토치로 만들기

In [3]:
pip install torch torchvision

Note: you may need to restart the kernel to use updated packages.


## **모델 (efficient + triplet)**

필요 라이브러리 설치

In [1]:
import torch

#신경망(Neural Network) 관련 기능을 제공하는 모듈. 
#레이어(layer), 활성화 함수(activation), 손실 함수(loss), 신경망 구성 요소 포함
import torch.nn as nn

import torchvision.models as models

from torchvision import transforms
from torch.utils.data import Dataset, DataLoader

#코사인 유사도 계산
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

from PIL import Image
import os
import random

efficient net -> 임베딩 벡터 변환 함수

In [ ]:
class EfficientNetEmbedding(nn.Module):
    def __init__(self, embedding_size=128):
        super().__init__()
        self.base_model = models.efficientnet_b0(pretrained=True)  # 사전학습 EfficientNet. imagenet 사용(논문과동일)
        self.features = self.base_model.features  # 분류기(fc) 제거, feature extractor 부분만 사용
        self.pool = nn.AdaptiveAvgPool2d(1)  # 마지막 feature map에 global average pooling
                                            # feature map을 한줄 벡터로 압축해야해서...
                                            
        self.embedding = nn.Linear(1280, embedding_size)  # 1280채널 → 임베딩 크기(128)로 축소
        self.l2_norm = nn.functional.normalize  # 임베딩 벡터 정규화 함수

    def forward(self, x):
        x = self.features(x)  # 이미지 특징 추출
        x = self.pool(x)  # 채널별 평균값으로 차원 축소
        x = torch.flatten(x, 1)  # 2D → 1D 벡터로 변환
        x = self.embedding(x)  # 임베딩 벡터 생성
        x = self.l2_norm(x, dim=1)  # 임베딩 벡터 정규화 (길이 1로)
        return x


In [ ]:
model = EfficientNetEmbedding(embedding_size=128) # ✅ 임베딩 바꾸기 가능. 현재 128

loss_fn = nn.TripletMarginLoss(margin=1.0)  # Triplet Loss 함수 (margin은 거리 차이 최소 기준)
#앵커와 음성 간 거리가 앵커와 양성 간 거리보다 최소 1.0 이상 더 커야 손실이 0이 되고 학습이 멈춤 (조건 만족)
#만약 두 거리 차이가 margin보다 작으면 손실이 양수이고, 모델은 차이를 늘리려고 학습함

optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

c:\Users\Administrator\.conda\envs\tf-env\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Administrator\.conda\envs\tf-env\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
# 3) Triplet Dataset 클래스 정의. 앵커, 양수, 음수 이미지 선택하는 함수.
class TripletDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform 
        # 곡 ID 폴더 리스트
        self.song_dirs = [os.path.join(root_dir, d) for d in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, d))]
        
        # 곡별 이미지 리스트 준비
        self.data = []
        for song_dir in self.song_dirs:
            images = [f for f in os.listdir(song_dir) if f.lower().endswith(('.png','.jpg','.jpeg'))]
            if len(images) >= 2:  # 양성 쌍 생성 가능하려면 2개 이상 필요
                self.data.append((song_dir, images))
        
    def __len__(self):
        # 전체 이미지 수 기준 (대략)
        return sum(len(images) for _, images in self.data)
    
    def __getitem__(self, idx):
        # 임의로 앵커 곡과 이미지 선택
        anchor_song_idx = random.randint(0, len(self.data) - 1)
        anchor_song_dir, anchor_images = self.data[anchor_song_idx]

        # 앵커 이미지 선택
        anchor_img_name = random.choice(anchor_images)
        # 양성 이미지(같은 곡 내 다른 이미지) 선택 (본인 제외)
        positive_img_name = anchor_img_name
        while positive_img_name == anchor_img_name:
            positive_img_name = random.choice(anchor_images)

        # 음성 곡과 이미지 선택 (다른 곡)
        negative_song_idx = anchor_song_idx
        while negative_song_idx == anchor_song_idx:
            negative_song_idx = random.randint(0, len(self.data) -1)
        negative_song_dir, negative_images = self.data[negative_song_idx]
        negative_img_name = random.choice(negative_images)

        # 이미지 로드 및 transform 적용
        anchor_img = Image.open(os.path.join(anchor_song_dir, anchor_img_name)).convert('RGB')
        positive_img = Image.open(os.path.join(anchor_song_dir, positive_img_name)).convert('RGB')
        negative_img = Image.open(os.path.join(negative_song_dir, negative_img_name)).convert('RGB')

        if self.transform:
            anchor_img = self.transform(anchor_img)
            positive_img = self.transform(positive_img)
            negative_img = self.transform(negative_img)

        return anchor_img, positive_img, negative_img

In [ ]:
# 4) 이미지 전처리 설정 (EfficientNet 사전학습 기준) ✅✅✅✅증강 처리 필요!!
transform = transforms.Compose([
    #전에 하던 증강이랑 똑같이 처리.
        transforms.RandomAffine(
        degrees=0,
        translate=(0.1, 0.1),
        scale=(0.9, 1.1)
    ),
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225]),
])

In [ ]:
# 5) 데이터셋과 데이터로더 생성
root_data_dir = "./data"  # ✅ 수정
triplet_dataset = TripletDataset(root_dir=root_data_dir, transform=transform)
data_loader = DataLoader(triplet_dataset, batch_size=32, shuffle=True, num_workers=4)

triplet loss 함수

In [7]:
def train_triplet(model, data_loader, optimizer, loss_fn, device):
    model.train()  # 모델을 학습 모드로 설정
    total_loss = 0  # 손실 합산용 변수 초기화
    
    for anchor, positive, negative in data_loader:  # 데이터로더에서 triplet 배치 단위로 불러오기
        anchor = anchor.to(device)      # 앵커 이미지 배치를 GPU/CPU에 올림
        positive = positive.to(device)  # 양성 이미지 배치를 GPU/CPU에 올림
        negative = negative.to(device)  # 음성 이미지 배치를 GPU/CPU에 올림

        optimizer.zero_grad()  # 이전 배치의 기울기 초기화
        
        # 각 배치에 대해 임베딩 벡터 생성
        anchor_embed = model(anchor)      
        positive_embed = model(positive)
        negative_embed = model(negative)
        
        # Triplet Loss 계산 (앵커-양성은 가깝게, 앵커-음성은 멀게)
        loss = loss_fn(anchor_embed, positive_embed, negative_embed)
        
        # 손실값을 기준으로 역전파 (모델 가중치 업데이트 방향 계산)
        loss.backward()
        
        # 옵티마이저로 가중치 업데이트
        optimizer.step()
        
        # 배치 손실값을 누적
        total_loss += loss.item()
    
    # 전체 데이터셋 평균 손실값 반환
    return total_loss / len(data_loader)



데이터를 batch 단위로 받아서 EfficientNet 모델로 임베딩 벡터를 뽑고

Triplet Loss 함수로 거리 차이를 계산해서 손실값을 구함

역전파로 모델 파라미터를 업데이트

## **갤러리, 테스트 임베딩 추출**

In [ ]:
def extract_embeddings(model, inputs, device, batch_size=64):
    """
    inputs: DataLoader 또는 Tensor(single or batch)
    """
    model.eval()
    embeddings = []

    with torch.no_grad():
        if isinstance(inputs, DataLoader): #갤러리 일 경우.
            for batch in inputs:
                batch = batch.to(device)
                emb = model(batch)
                embeddings.append(emb.cpu().numpy())
            embeddings = np.vstack(embeddings)
        else: #테스트일 경우.
            # 단일 또는 소량 이미지 텐서 처리
            inputs = inputs.to(device)
            emb = model(inputs)
            embeddings = emb.cpu().numpy()

    return embeddings

## **top-k 추천**

In [ ]:
def recommend_topk(query_embedding, gallery_embeddings, gallery_ids, topk=5):
    sims = cosine_similarity(query_embedding.reshape(1, -1), gallery_embeddings).flatten()
    topk_idx = sims.argsort()[::-1][:topk]
    return [(gallery_ids[i], sims[i]) for i in topk_idx]


In [ ]:
# 9) 학습 실행 예시
if __name__ == "__main__":
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    num_epochs = 10
    for epoch in range(num_epochs):
        avg_loss = train_triplet(model, data_loader, optimizer, loss_fn, device)
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.4f}")
        
        # 필요하면 체크포인트 저장
        torch.save(model.state_dict(), f"model_epoch_{epoch+1}.pth")

    # 10) 갤러리 임베딩 미리 추출 및 저장 예시
    gallery_loader = DataLoader(triplet_dataset, batch_size=64, shuffle=False, num_workers=4)
    gallery_embeddings = extract_embedding(model, gallery_loader, device)
    np.save("gallery_embeddings.npy", gallery_embeddings)
    
    # 11) 테스트 쿼리 임베딩과 추천 예시
    # (테스트용 단일 이미지 예시)
    test_img_path = "./test_query.png"  # 사용자 쿼리 이미지 경로로 변경하세요
    test_img = Image.open(test_img_path).convert('RGB')
    test_img_tensor = transform(test_img).unsqueeze(0).to(device)  # 배치 차원 추가
    
    model.eval()
    with torch.no_grad():
        query_embedding = model(test_img_tensor).cpu().numpy()
    
    gallery_ids = [d for d, _ in triplet_dataset.data]  # 곡 ID 리스트 (폴더명)
    recommendations = recommend_topk(query_embedding, gallery_embeddings, gallery_ids, topk=5)
    print("추천 결과:", recommendations)